In [1]:
from fastai.text import *
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
import sentencepiece as spm
import re
import pdb

In [2]:
import fastai, torch
fastai.__version__ , torch.__version__

('1.0.57', '1.0.0')

In [3]:
torch.cuda.set_device(0)

In [4]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value)  
    torch.manual_seed(seed_value)  
    random.seed(seed_value)
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

In [5]:
random_seed(42, True)

In [6]:
!pwd

/data/home/ubuntu/gaurav/in/nlp-for-malyalam/classification


In [7]:
path = Path('./')

In [8]:
df_train = pd.read_csv(path/'../../classification_public_datasets/inltk-headlines/ml/ml-train.csv', header=None)
df_train.head()

,0,1
0,business,ജോലിയില്‍ നിന്ന് ഒരു ബ്രേക്ക് എടുക്കുന്നതിനു മ...
1,business,കമ്ബോളങ്ങള്‍ കരടിയുടെ പിടിയില്‍
2,business,കൊച്ചി മെട്രോയുടെ ബ്രാന്‍ഡ് അംബാസിഡറായി നടന്‍ ...
3,business,"ഇന്ധനവിലയില്‍ വീണ്ടും വര്‍ദ്ധനവ്, പെട്രോളിന് 1..."
4,sports,ഫെഡറേഷന്‍ കപ്പ‌് അത‌്‌ലറ്റിക‌്സിന‌് ഇന്ന‌് തുട...


In [9]:
Counter(list(df_train[0]))

Counter({'business': 1551, 'entertainment': 1759, 'sports': 1726})

In [10]:
df_valid = pd.read_csv(path/'../../classification_public_datasets/inltk-headlines/ml/ml-valid.csv', header=None)
df_valid.head()

,0,1
0,business,"ട്രെയിന്‍ യാത്രയില്‍ ഇനി കുലുക്കം കുറയും, ജെര്..."
1,sports,പാ​ലാ സെ​ന്‍റ് തോ​മ​സ് ചാ​മ്ബ്യന്മാ​ര്‍
2,sports,ഓസ്‌ട്രേലിയയ്‌ക്കെതിരെ ഇനി ധോണിയില്ല; ലോകകപ്പി...
3,sports,ടെസ്റ്റിന് വേഗം കൂട്ടാന്‍ എം.സി.സി
4,sports,ഓള്‍ ഇംഗ്ലണ്ട് ബാഡ്മിന്റണില്‍ ശ്രീകാന്തും പുറത...


In [11]:
df_test = pd.read_csv(path/'../../classification_public_datasets/inltk-headlines/ml/ml-test.csv', header=None)
df_test.head()

,0,1
0,sports,ഇഞ്ചുറി ടൈം പെനാല്‍റ്റിയില്‍ എഫ് സി പോര്‍ട്ടോ
1,entertainment,ആമിര്‍ ഖാന്റെ ഏറ്റവും പുതിയ ചിത്രം ലാല്‍ സിങ് ...
2,sports,ഐ പി എല്ലിന് മുന്‍പായി ഓസ്‌ട്രേലിയന്‍ ടീമിനൊപ്...
3,business,സാമ്ബത്തിക ജീവിതം സുരക്ഷിതമാക്കണോ? ഈ അഞ്ച് ശീല...
4,business,എല്‍ഇഡി ബള്‍ബുകള്‍ ലഭ്യമാക്കും; പദ്ധതിയുടെ രജി...


In [12]:
df_train.shape, df_valid.shape, df_test.shape

((5036, 2), (630, 2), (630, 2))

In [13]:
cutoff = int(0.1*len(df_train))

In [14]:
# from inltk.inltk import get_similar_sentences, tokenize
# import warnings
# warnings.filterwarnings("ignore")
# aug_sen = []
# aug_label = []
# for index, row in df_train[len(aug_sen)//2:cutoff].iterrows():
#     augs = get_similar_sentences(row[1], no_of_variations=2, language_code='ml', degree_of_aug=0.3)
#     aug_sen.extend(augs)
#     aug_label.extend([row[0]]*len(augs))
# df_aug = pd.DataFrame({0: aug_label, 1: aug_sen })
# df_aug.head()
# df_aug.to_csv('inltk_headlines_ml_train_aug.csv', index=False)

In [15]:
df_train_aug = pd.read_csv('inltk_headlines_ml_train_aug.csv')
df_train_aug.head()

,0,1
0,business,ജോലിയിലായിരുന്നു് നിന്ന് ഒരു ബ്രേക്ക് എടുക്കുന...
1,business,ജോലിയിലും നിന്നു മറ്റൊരു ബ്രേക്ക് എടുക്കുന്നതി...
2,business,കമ്ബുളങ്ങള് കരടിയയുടെപിടിയില്
3,business,കുമ്ബോളങ്ങള് കരടയുടെയും പിടിയില്
4,business,കൊച്ചി ടാന്റലംയുടെ ബ്രാന്മാർ ഡുകൾ അംബാകസിഡിറായ...


In [16]:
df_train = pd.DataFrame({0: list(df_train[0])[:cutoff] + list(df_train_aug['0'])[:2*cutoff], 1: list(df_train[1])[:cutoff] + list(df_train_aug['1'])[:2*cutoff]})
df_train.shape

(1509, 2)

In [17]:
df_train[df_train[0].isnull()].shape, df_valid[df_valid[0].isnull()].shape, df_test[df_test[0].isnull()].shape

((0, 2), (0, 2), (0, 2))

In [18]:
label_cols = [0]

In [19]:
class MalyalamTokenizer(BaseTokenizer):
    def __init__(self, lang:str):
        self.lang = lang
        self.sp = spm.SentencePieceProcessor()
        self.sp.Load(str('./../../models/malayalam/tokenizer/malyalam_lm.model'))
        
    def tokenizer(self, t:str) -> List[str]:
        return self.sp.EncodeAsPieces(t)

In [20]:
sp = spm.SentencePieceProcessor()
sp.Load(str('./../../models/malayalam/tokenizer/malyalam_lm.model'))
itos = [sp.IdToPiece(int(i)) for i in range(10000)]

In [21]:
# 10,000 is the vocab size that we chose in sentencepiece
malyalam_vocab = Vocab(itos)

In [22]:
tokenizer = Tokenizer(tok_func=MalyalamTokenizer, lang='ml')

In [23]:
tokenizer.special_cases

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep']

In [24]:
data_lm = TextLMDataBunch.from_df(path=path, train_df=df_train, valid_df=df_valid, test_df=df_test, tokenizer=tokenizer, vocab=malyalam_vocab)

In [25]:
data_lm.show_batch()

idx,text
0,"് ▁വീണ്ടും ▁വര ് ▁ ദ്ധ ന വ് , ▁പെട്രോ ള ിന് ▁14 ▁പൈ സ യും ▁ഡീ സ ലി ന് ▁15 ▁പൈ സ യും ▁വര ് ▁ ദ്ധ ിച്ചു ▁ x x bo s ▁ഫെഡറ േഷന ് ▁കപ്പ ▁ ് ▁അത ▁ ് ▁ല റ്റി ക ▁ ് സി ന ▁ ് ▁ഇന്ന ▁ ് ▁തുടക്കം ▁ x x bo s ▁ഓഹരി ▁സൂചി ക കള ് ▁നേട്ട ത്തില ് ▁"
1,"ിലേക്ക് ▁ x x bo s ▁ഹാ ട്രി ക്കു മായി ▁ക്രി സ്റ്റ ിയ ാനോ ▁അവതരിച്ച ു ; ▁അത് ▁ല റ്റി കോ യെ ▁ത കര ് ▁ ത്ത് ▁യുവ ന്റ സ് ▁ചാ മ ് ബ് യ ന് ▁സ് ▁ലീഗ് ▁ക്വാ ര ് ▁ ട്ട റി ല് ▁ x x bo s ▁"" താ ന് ▁ആയിരുന്നു ▁പി ▁എസ് ▁ജി ▁പരിശീലക ന് ▁എ ങ്ക ില ് ▁പാല ത്തില ് ▁നിന്ന് ▁എടുത്ത ് ▁ചാ"
2,ാ വിന് ▁വിധി ച്ച ▁ശിക്ഷ ▁ഇങ്ങനെ ▁ x x bo s ▁വീണ്ടും ▁ഹോ ട്ട ായി ▁റായ് ▁ലക്ഷ ് ▁മി . . <unk> ▁ x x bo s ▁ഇ ഞ്ചു റി ▁ടൈ മി ല് ▁ചെ ല് ▁സി യുടെ ▁രക്ഷ ക നായി ▁ഹ സാര ് ▁ ഡ് . ▁ x x bo s ▁സ്വന്തം ▁നാ ട്ടി ല് ▁ആദ്യമായി ▁പരമ ് ബ ര ▁നഷ്ടപ്പെട്ട ് ▁കോ ഹ് ▁ലി ▁ടീം <unk> ▁ x x
3,"; ▁പൈതൃക ▁സംരക്ഷണ ത്തിനും ▁നവീകരണ ത്തിനു മായി ▁25 ▁കോടി യുടെ ▁പദ്ധതി , ▁നൂറ ാം ▁ വാര ് ▁ ഷിക ത്തില ് ▁ഹോട്ടല ിന് ▁പഞ്ച നക്ഷത്ര ▁പദവി യും ▁ x x bo s ▁കെ ▁എ ▁സ് ▁ആ ▁ര ് ▁ടി ▁സി ▁ഇ ▁ല ▁ക് ▁ ട്രി ▁ക് ▁ബ ▁സ് : ▁എ ▁റ ▁ ണാ ▁കു ▁ ള ▁ ത്തും ▁ഹ ▁ രി ▁ പ്പാ ▁ ട്ടു ം ▁ചാ ▁ര ് ▁ജി"
4,"ൈ സ്റ്റ് ; ▁പൃഥ്വി രാജ് , ▁ഇന്ദ്ര ജി ത് , ▁ഫഹദ് : ▁വ മ ് ബ ന് ▁താര നിര യുമായി ▁ലി ജോ ▁ജോ സ് ▁പല്ല ി ശ്ശേരി യുടെ ▁പുതിയ ▁ചിത്രം ▁ x x bo s ▁ല ച്ചു വും ▁ശിവ യു മല്ല ▁അ നി ഘ യാണ് ▁ശരിക്കു മുള്ള ▁ചേ ച്ചി <unk> ▁ഉപ്പു ം ▁മുള ക ിലെ ▁പാറ ു ക്കുട്ടി ▁ആര ാണെന്ന റിയ ു മോ <unk> ▁കാണ ൂ <unk> ▁ x x"


In [26]:
awd_lstm_config = awd_lstm_lm_config.copy()
awd_lstm_config['n_hid'] = 1150
learn = language_model_learner(data_lm, arch=AWD_LSTM, drop_mult=0.3, config=awd_lstm_config, pretrained=False)

In [27]:
# Loading the pretrained language model on malyalam wikipedia
learn.load('../../../models/malayalam/lm/ULMFiT/third_ml_lm', with_opt=True)

LanguageLearner(data=TextLMDataBunch;

Train: LabelList (1509 items)
x: LMTextList
▁ x x bo s ▁ജോലി യില ് ▁നിന്ന് ▁ഒരു ▁ബ്ര േക്ക് ▁എടുക്ക ുന്നതിനു ▁മു ന് ▁പ ് . .,▁ x x bo s ▁ക മ ് ബോ ള ങ്ങള ് ▁കര ടി യുടെ ▁പിടി യില ്,▁ x x bo s ▁കൊച്ചി ▁മെട്രോ യുടെ ▁ബ്രാ ന് ▁ ഡ് ▁അംബ ാ സി ഡ റായി ▁നട ന് ▁സുരേഷ് ▁ഗോപി യെ ▁നിയമ ിച്ചു,▁ x x bo s ▁ഇന്ധന വില യില ് ▁വീണ്ടും ▁വര ് ▁ ദ്ധ ന വ് , ▁പെട്രോ ള ിന് ▁14 ▁പൈ സ യും ▁ഡീ സ ലി ന് ▁15 ▁പൈ സ യും ▁വര ് ▁ ദ്ധ ിച്ചു,▁ x x bo s ▁ഫെഡറ േഷന ് ▁കപ്പ ▁ ് ▁അത ▁ ് ▁ല റ്റി ക ▁ ് സി ന ▁ ് ▁ഇന്ന ▁ ് ▁തുടക്കം
y: LMLabelList
,,,,
Path: .;

Valid: LabelList (630 items)
x: LMTextList
▁ x x bo s ▁ട്രെയിന ് ▁യാത്ര യില ് ▁ഇനി ▁കുല ു ക്കം ▁കുറയ ും , ▁ജെ ര ് ▁ ക്ക ിങ് ▁ഒഴിവാക്ക ുന്നതിനുള്ള ▁നൂതന ▁സാങ്കേതിക ▁വിദ്യ ▁പ്രീ മിയ ം ▁ട്രെയിന ുകളില ്,▁ x x bo s ▁പാ ▁ലാ ▁സെ ▁ന് ▁റ ് ▁തോ ▁മ ▁സ് ▁ചാ ▁മ ് ബ് യ ന് മാ ▁ര ്,▁ x x bo s ▁ഓ സ് ▁ട്ര േലിയ യ് ▁ ക്കെതിരെ ▁ഇനി ▁ധ ോ ണിയ ില്ല ; ▁ലോകകപ്പ ിന് ▁മു മ ് ബ് ▁ഋഷഭ ് ▁പന്ത ിന് ▁സു വര ് ▁ ണാ വസ രം <unk>,▁ x x bo s ▁ടെസ്റ്റ ിന് ▁ വേഗ ം ▁കൂട്ട ാന ് ▁എം

In [28]:
# Fine tuning the prtrained LM on current dataset

In [29]:
learn.freeze()

In [30]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,6.446610,5.327024,0.239821,00:01


In [31]:
learn.unfreeze()

In [32]:
learn.fit_one_cycle(5, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,5.909966,5.059901,0.268973,00:01
1,5.647250,4.661511,0.318795,00:01
2,5.393860,4.494516,0.334554,00:01
3,5.201211,4.426492,0.340938,00:01
4,5.066526,4.413632,0.342098,00:01


In [33]:
learn.predict('മലയാള ികളായ ▁വിമാന യാത്ര ക്കാര',n_words=10)

'മലയാള ികളായ ▁വിമാന യാത്ര ക്കാര ന് ▁അത ിനൊപ്പം ▁വന്ന ് ▁ഭക്ഷണ മു റി ▁വി മാന'

In [34]:
learn.save_encoder('fine_tuned_enc')

In [35]:
data_clas = TextClasDataBunch.from_df(path=path, train_df=df_train, valid_df=df_valid, test_df=df_test, tokenizer=tokenizer, vocab=malyalam_vocab, bs=16)

In [36]:
data_clas.show_batch()

text,target
▁ x x bo s ▁റാ ഞ്ചി ▁ഏകദിന ത്തിലായി ് ▁ഉ സിന്റെ മാന ് ▁ഖു ്വാ ജന യ്ക്ക് : ▁/ ▁/ ▁ ; ▁മോശ വും ▁ഫോ മി ല് ▁നിന്ന് ▁അഗ്നിശമന വും ▁നേടി ▁ത കർ ് ▁ ത്ത ടിച്ച ് നീലക്കടുവ ണ് ▁ഫി ഞ്ച് ; ▁വെടി ക്കെ ട്ട് ▁അരങ്ങേറ്റം ▁മുതല ാ ക്കാതെ ും ▁കഴിയ ുന്നതുവരെ ▁ഓ സീ സ് ▁. ▁പാകിസ്താൻ ക്ക് ▁3 14 ▁മസ് ണ് ▁ഇൻ ▁വിജയിച്ച ലക്ഷ ്യം ▁. ▁മാ,sports
▁ x x bo s ▁ഇവയെ ളു േ ▁വീട്ടിലേക്ക ും ▁കൂട്ടി യാ ലോ ▁എന്നാണ് ▁ രണ ും ▁വീ റിന്റെ ▁ചോദ്യം x x un k ▁ലെ ണ്ട ന ിലും ▁ഷ ൂ ട്ട ിനെ ത്തും മി ് ബോ ളു ് ▁കാണ ാമെന്ന് ▁തോന്നിയ ാല ് ▁ഇ ങ്ങോട്ട ് ▁എത്തിയ ാല ് ▁ബുദ്ധിമുട്ട യെന്ന് ▁ദീപ ീയ യുടെ ▁ഉപദേശ വും ; ▁ലോ ഡ വും ▁ട ് യൊ ▁സാ ഡ് ▁സ് ▁വാ ക് ▁സ് ▁അക്കാഡമി,entertainment
▁ x x bo s ▁ഇവ ള േ ▁വീട്ടിലേക്ക ് ▁ബന്ധപ്പെടുത്തി യാ ലോ ▁എന്നാണ് ▁ രണ ് ▁വീ റിന് ▁സ്വാഗതം x x un k ▁ല ണ്ട് നില ് ▁ഷ ാവൂ ട്ട ിനെ ത്തും മ ് ബോ ള ും ▁കാണ ണമെന്ന് ▁തോന്നിയ ാല ് ▁ഇ ങ്ങോട്ട ും ▁പ്രവേശിച്ച ാക് ▁മതി യെന്ന് ദീപ ിക യുടെയും ▁ക്ഷണ ം ; ▁ലോ ഡ വും ▁ട ് യു ▁സാ ഡ് ▁സ് ▁ടാ ക് ▁സ് ▁മ്യൂസിയ ത്തിലായി,entertainment
▁ x x bo s ▁അഗ്നിശമന ▁സംഘം ങ്ങളുടെ ▁പേര ിനൊപ്പം ▁‘ ബാങ്ക ു ' ▁എന്ന് ▁ചേര ു ▁ ത്തു ിട്ടു ണ്ട െ ങ്ക ിലായി ് ▁നിക്ഷേപ ങ്ങള ് ▁അ ക്ക് ▁നികുതി ▁ഭ ല് ▁ക ണമെന്നും ▁കാ ദ ായ ▁നി ക ത ▁വകുപ്പ ് ; നിക്ഷേപ ത്തിന്റെ ▁പലിശ യിലായിരുന്നു ് ▁നിന്ന് ▁നികുതി ▁ഈ പഞ്ചാബി ക്കുന്നതിന് ് ▁ജില്ലാ ▁സഹകരണ ▁അക്കൗണ്ടിംഗ ുകളില ് ▁ ക്ക് നിര ് ▁ദ് ദേശം ; ▁പാളി സ നു,business
▁ x x bo s ▁സഹകരണ ▁സംഘ ങ്ങളുടെ ▁പേര ിനൊപ്പം ▁' ബാങ്ക ് ' ▁എന്ന് ▁ചേര ് ▁ ത്ത ിട്ടു ണ്ട െ ങ്ക ില ് ▁നിക്ഷേപ ങ്ങള ് ▁ ക്ക് ▁നികുതി ▁ന ല് ▁ക ണമെന്ന് ▁ആ ദ ായ ▁നി ക തി ▁വകുപ്പ ് ; ▁നിക്ഷേപ ത്തിന്റെ ▁പലിശ യില ് ▁നിന്നും ▁നികുതി ▁ഈ ടാ ക്കാന ് ▁ജില്ലാ ▁സഹകരണ ▁ബാങ്ക ുകള ് ▁ ക്ക് ▁നിര ് ▁ദ് ദേശം ; ▁ലൈ സ ന്,business


In [37]:
del awd_lstm_config['tie_weights']
del awd_lstm_config['out_bias']

In [38]:
learn = text_classifier_learner(data_clas, arch=AWD_LSTM, drop_mult=0.5, config=awd_lstm_config)

In [39]:
learn.load_encoder('fine_tuned_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (1509 items)
x: TextList
▁ x x bo s ▁ജോലി യില ് ▁നിന്ന് ▁ഒരു ▁ബ്ര േക്ക് ▁എടുക്ക ുന്നതിനു ▁മു ന് ▁പ ് . .,▁ x x bo s ▁ക മ ് ബോ ള ങ്ങള ് ▁കര ടി യുടെ ▁പിടി യില ്,▁ x x bo s ▁കൊച്ചി ▁മെട്രോ യുടെ ▁ബ്രാ ന് ▁ ഡ് ▁അംബ ാ സി ഡ റായി ▁നട ന് ▁സുരേഷ് ▁ഗോപി യെ ▁നിയമ ിച്ചു,▁ x x bo s ▁ഇന്ധന വില യില ് ▁വീണ്ടും ▁വര ് ▁ ദ്ധ ന വ് , ▁പെട്രോ ള ിന് ▁14 ▁പൈ സ യും ▁ഡീ സ ലി ന് ▁15 ▁പൈ സ യും ▁വര ് ▁ ദ്ധ ിച്ചു,▁ x x bo s ▁ഫെഡറ േഷന ് ▁കപ്പ ▁ ് ▁അത ▁ ് ▁ല റ്റി ക ▁ ് സി ന ▁ ് ▁ഇന്ന ▁ ് ▁തുടക്കം
y: CategoryList
business,business,business,business,sports
Path: .;

Valid: LabelList (630 items)
x: TextList
▁ x x bo s ▁ട്രെയിന ് ▁യാത്ര യില ് ▁ഇനി ▁കുല ു ക്കം ▁കുറയ ും , ▁ജെ ര ് ▁ ക്ക ിങ് ▁ഒഴിവാക്ക ുന്നതിനുള്ള ▁നൂതന ▁സാങ്കേതിക ▁വിദ്യ ▁പ്രീ മിയ ം ▁ട്രെയിന ുകളില ്,▁ x x bo s ▁പാ ▁ലാ ▁സെ ▁ന് ▁റ ് ▁തോ ▁മ ▁സ് ▁ചാ ▁മ ് ബ് യ ന് മാ ▁ര ്,▁ x x bo s ▁ഓ സ് ▁ട്ര േലിയ യ് ▁ ക്കെതിരെ ▁ഇനി ▁ധ ോ ണിയ ില്ല ; ▁ലോകകപ്പ ിന് ▁മു മ ് ബ് ▁ഋഷഭ ് ▁പന്ത ിന് ▁സു വര ് ▁ ണാ വസ രം <unk>,▁ x x bo s ▁ടെസ

In [40]:
learn.freeze()

In [41]:
learn.loss_func.func

CrossEntropyLoss()

In [42]:
mcc = MatthewsCorreff()

In [43]:
learn.metrics = [mcc, accuracy]

In [44]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,matthews_correff,accuracy,time
0,0.736228,0.575900,0.659136,0.773016,00:02


In [45]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,matthews_correff,accuracy,time
0,0.467785,0.475864,0.718939,0.812698,00:02


In [46]:
learn.save('second-full')

In [47]:
learn.unfreeze()
learn.fit_one_cycle(5, 1e-3, callbacks=[callbacks.SaveModelCallback(learn, every='improvement', monitor='accuracy', name='final')])

epoch,train_loss,valid_loss,matthews_correff,accuracy,time
0,0.253450,0.442634,0.764312,0.842857,00:04
1,0.138819,0.395225,0.783676,0.855556,00:04
2,0.097203,0.423959,0.766831,0.844444,00:04
3,0.071407,0.423179,0.771765,0.847619,00:04
4,0.057329,0.435254,0.768512,0.844444,00:04


Better model found at epoch 0 with accuracy value: 0.8428571224212646.
Better model found at epoch 1 with accuracy value: 0.855555534362793.


In [48]:
learn.load('final')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (1509 items)
x: TextList
▁ x x bo s ▁ജോലി യില ് ▁നിന്ന് ▁ഒരു ▁ബ്ര േക്ക് ▁എടുക്ക ുന്നതിനു ▁മു ന് ▁പ ് . .,▁ x x bo s ▁ക മ ് ബോ ള ങ്ങള ് ▁കര ടി യുടെ ▁പിടി യില ്,▁ x x bo s ▁കൊച്ചി ▁മെട്രോ യുടെ ▁ബ്രാ ന് ▁ ഡ് ▁അംബ ാ സി ഡ റായി ▁നട ന് ▁സുരേഷ് ▁ഗോപി യെ ▁നിയമ ിച്ചു,▁ x x bo s ▁ഇന്ധന വില യില ് ▁വീണ്ടും ▁വര ് ▁ ദ്ധ ന വ് , ▁പെട്രോ ള ിന് ▁14 ▁പൈ സ യും ▁ഡീ സ ലി ന് ▁15 ▁പൈ സ യും ▁വര ് ▁ ദ്ധ ിച്ചു,▁ x x bo s ▁ഫെഡറ േഷന ് ▁കപ്പ ▁ ് ▁അത ▁ ് ▁ല റ്റി ക ▁ ് സി ന ▁ ് ▁ഇന്ന ▁ ് ▁തുടക്കം
y: CategoryList
business,business,business,business,sports
Path: .;

Valid: LabelList (630 items)
x: TextList
▁ x x bo s ▁ട്രെയിന ് ▁യാത്ര യില ് ▁ഇനി ▁കുല ു ക്കം ▁കുറയ ും , ▁ജെ ര ് ▁ ക്ക ിങ് ▁ഒഴിവാക്ക ുന്നതിനുള്ള ▁നൂതന ▁സാങ്കേതിക ▁വിദ്യ ▁പ്രീ മിയ ം ▁ട്രെയിന ുകളില ്,▁ x x bo s ▁പാ ▁ലാ ▁സെ ▁ന് ▁റ ് ▁തോ ▁മ ▁സ് ▁ചാ ▁മ ് ബ് യ ന് മാ ▁ര ്,▁ x x bo s ▁ഓ സ് ▁ട്ര േലിയ യ് ▁ ക്കെതിരെ ▁ഇനി ▁ധ ോ ണിയ ില്ല ; ▁ലോകകപ്പ ിന് ▁മു മ ് ബ് ▁ഋഷഭ ് ▁പന്ത ിന് ▁സു വര ് ▁ ണാ വസ രം <unk>,▁ x x bo s ▁ടെസ

In [49]:
from sklearn.metrics import accuracy_score, matthews_corrcoef
df_dict = {'query': list(df_test[1]), 'actual_label': list(df_test[0]), 'predicted_label': ['']*df_test.shape[0]}
all_nodes = list(set(df_train[0]))
for node in all_nodes:
    df_dict[node] = ['']*df_test.shape[0]
    
i2c = {}
for key, value in learn.data.c2i.items():
    i2c[value] = key
    
df_result = pd.DataFrame(df_dict)
preds = learn.get_preds(ds_type=DatasetType.Test, ordered=True)
for index, row in df_result.iterrows():
    for node in all_nodes:
        row[node] = preds[0][index][learn.data.c2i[node]].item()
    row['predicted_label'] = i2c[np.argmax(preds[0][index]).data.item()]
df_result.head()

,query,actual_label,predicted_label,sports,entertainment,business
0,ഇഞ്ചുറി ടൈം പെനാല്‍റ്റിയില്‍ എഫ് സി പോര്‍ട്ടോ,sports,sports,0.962045,0.0160935,0.0218611
1,ആമിര്‍ ഖാന്റെ ഏറ്റവും പുതിയ ചിത്രം ലാല്‍ സിങ് ...,entertainment,entertainment,0.358346,0.626584,0.0150703
2,ഐ പി എല്ലിന് മുന്‍പായി ഓസ്‌ട്രേലിയന്‍ ടീമിനൊപ്...,sports,sports,0.925773,0.00352715,0.0706998
3,സാമ്ബത്തിക ജീവിതം സുരക്ഷിതമാക്കണോ? ഈ അഞ്ച് ശീല...,business,entertainment,0.0136561,0.661898,0.324446
4,എല്‍ഇഡി ബള്‍ബുകള്‍ ലഭ്യമാക്കും; പദ്ധതിയുടെ രജി...,business,sports,0.463686,0.213866,0.322448


In [50]:
accuracy_score(df_result['actual_label'], df_result['predicted_label'])

0.8428571428571429

In [51]:
matthews_corrcoef(df_result['actual_label'], df_result['predicted_label'])

0.7636431681809049

In [52]:
df_result.to_csv('inltk_headlines_ml_with_aug.csv', index=False)